In [ ]:
import os
os.environ["JAX_ENABLE_X64"] = "True"
import matplotlib.pyplot as plt
import h5py
import numpy as np
import networkx as nx
import yaml
from numpy.typing import NDArray
from scipy.special import entr
from sklearn.cluster import SpectralClustering
from qem import QuantumState
from quantum_annealing.src.energy_function import EnergyFunction
from quantum_annealing.src.scheduler import get_scheduler
from quantum_annealing.src.exact_quantum_annealer import run_exact_quantum_annealer

plt.rcParams.update({
    "text.usetex": True,
    "font.family": "Helvetica",
    "font.size": 20,
})

def get_adj_matrix(
    nodes_number: int,
    pairs: NDArray,
) -> NDArray:
    adj_matrix = np.zeros((nodes_number, nodes_number), dtype=np.float64)
    for pair in pairs:
        adj_matrix[pair[0], pair[1]] = 1
        adj_matrix[pair[1], pair[0]] = 1
    return adj_matrix

script_dir = os.getcwd()
# ---------- here is the path to the results directory ----------------------
# here is the path to the results directory
# ---------------------------------------------------------------------------
results_dir_path = f"{script_dir}/outputs/qbp/exact_validation/20/2024-05-07-07-00-44"
results_path = f"{results_dir_path}/result"
config_path = f"{results_dir_path}/.hydra/config.yaml"
with open(config_path, "r") as f:
    raw_config = f.read()
config = yaml.safe_load(raw_config)
schedule = eval(config["quantum_annealing_schedule"]["schedule"])
layers_number = int(config["quantum_annealing_schedule"]["steps_number"])
time_step = float(config["quantum_annealing_schedule"]["total_time_step_size"])
time = np.arange(layers_number) * time_step
with h5py.File(results_path, "r") as f:
    circuit = nx.Graph()
    fields = np.array(f["fields"])
    pairs = np.array(f["coupled_spin_pairs"])
    nodes_number = fields.shape[0]
    edges_number = pairs.shape[0]
    coupling_amplitudes = np.array(f["coupling_amplitudes"])
    truncation_errors = np.array(f["truncation_errors"])
    entropies = np.array(f["entropies"])
    entropies = entropies.reshape((-1, edges_number))
energy_function = EnergyFunction(
    coupling_amplitudes,
    pairs,
    fields,
)
sc = SpectralClustering(
    affinity='precomputed',
    random_state=0,
    n_clusters=2,
)
adj_matrix = get_adj_matrix(nodes_number, pairs)
clustering = sc.fit(adj_matrix).labels_
subdens_positions = tuple(map(lambda x: x[0], filter(lambda x: x[1] == 1, enumerate(clustering))))
positions_number = len(subdens_positions)
print(positions_number)
cut_entropy = np.zeros((entropies.shape[0],))
cut_size = 0
for pair, entropy in zip(pairs, entropies.T):
    if clustering[pair[0]] != clustering[pair[1]]:
        cut_entropy += entropy
        cut_size += 1
schedule = get_scheduler(time_step, schedule, layers_number)
exact_entropies = []
def get_exact_entropy(state: QuantumState):
    dens = state.dens_large(subdens_positions)
    dens = dens.reshape((2 ** positions_number, 2 ** positions_number))
    exact_entropies.append(entr(np.maximum(np.linalg.eigvalsh(dens), 0.)).sum())
quantum_annealing_results = run_exact_quantum_annealer(
    energy_function,
    schedule,
    np.array(42),
    False,
    False,
    get_exact_entropy,
)
fig_entropy, ax = plt.subplots(1, 1)
ax.plot(time, cut_entropy, 'b', label=r"${\rm Approximate}$")
ax.plot(time, exact_entropies, 'r--', label=r"${\rm Exact}$")
ax.set_ylabel(r"$S_A$")
ax.set_xlabel(r"${\rm Time}$")
ax.legend(frameon=False)

In [ ]:
fig_entropy.savefig(f"{results_dir_path}/entropy_validation.pdf",  bbox_inches="tight")